In [3]:
pip install pandas scikit-learn  imbalanced-learn  xgboost pandas scikit-learn


  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached imbalanced_learn-0.13.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached xgboost-2.1.4-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scipy-1.15.2-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached sklearn_compat-0.1.3-py3-none-any.whl.metadata (18 kB)
Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl (11.1 MB)
Using cached imbalanced_learn-0.13.0-py3-none-any.whl (238 kB)
Using cached xgboost-2.1.4-py3-none-win_amd64.whl (124.9 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cach

In [2]:
pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2


  Using cached torch-2.0.1-cp310-cp310-win_amd64.whl.metadata (23 kB)
  Using cached torchvision-0.15.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached torchaudio-2.0.2-cp310-cp310-win_amd64.whl.metadata (1.2 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.2.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached pillow-11.1.0-cp310-cp310-win_amd64.whl.metadata (9.3 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.

In [4]:
import flwr as fl
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
# Load the dataset
df = pd.read_csv("farmighamdataset.csv")

In [ ]:
# Define target column
target_column = 'TenYearCHD'

# Separate majority and minority classes
df_majority = df[df[target_column] == 0]
df_minority = df[df[target_column] == 1]

# Upsample the minority class
df_minority_upsampled = resample(df_minority,replace=True,    # Sample with replacement
n_samples=len(df_majority),random_state=42)  


# Combine majority and upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
#Print new class distribution
print("Balanced Class Distribution:\n", df_balanced[target_column].value_counts())

In [ ]:
# Store accuracies
client_accuracies = []

# Train a Random Forest model on each client
for client_id, client_df in enumerate(client_splits, start=1):  # Ensure client_id is 1,2,3
    # Split client data into features and labels
    X_client = client_df.drop(columns=[target_column])
    y_client = client_df[target_column]

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_client, y_client, test_size=0.2, random_state=42)

    # Ensure there is training data
    print(f"\nClient {client_id}: Train size = {len(X_train)}, Test size = {len(X_test)}")

    if len(X_train) == 0 or len(X_test) == 0:
        print(f"Client {client_id} has insufficient data for training. Skipping...")
        continue

    # Train a Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Store accuracy
    client_accuracies.append((client_id, accuracy))
    print(f"Client {client_id}: Accuracy = {accuracy:.4f}")  # Print directly inside loop



In [ ]:
# Print all client accuracies at the end
print("\nFinal Client Accuracies:")
for client_id, acc in client_accuracies:
    print(f"Client {client_id}: Accuracy = {acc:.4f}")

In [5]:
import flwr as fl
import numpy as np

class SimpleClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        params = np.random.rand(2, 2)  # Random 2x2 matrix
        print(f"📤 Sending parameters: {params}")
        return [params]

    def fit(self, parameters, config):
        print(f"📥 Received parameters from server: {parameters}")
        return parameters, 1, {}

    def evaluate(self, parameters, config):
        print("📊 Evaluating model")
        loss = float(np.random.rand())  # Simulated loss
        accuracy = float(np.random.rand())  # Simulated accuracy
        return loss, 10, {"accuracy": accuracy, "loss": loss}

# Start client (use old API but expect warnings)
fl.client.start_client(
    server_address="127.0.0.1:8080",
    client=SimpleClient().to_client(),
)


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: train message d0600f53-ec04-43b1-9cb9-ce721db25c6f
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 9f60107a-2994-46ed-b05b-78f95a5f74d9
INFO :      Sent reply
INFO :      
INFO :      Received: train message 92cca11d-f7d8-4644-a130-04d2863da394
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 952282c9-3fb1-41fa-a0f9-d4b843af063f
INFO :      Sent reply
INFO :      
INFO :      Received: train message 70393937-8a0b-4aa4-8035-5842794a77fe
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message c916d71c-0004-4e43-a9d7

📥 Received parameters from server: [array([[0.36020452, 0.3932013 ]], dtype=float32), array([-0.3815017], dtype=float32)]
📊 Evaluating model
📥 Received parameters from server: [array([[0.37044483, 0.40848482]], dtype=float32), array([-0.3310699], dtype=float32)]
📊 Evaluating model
📥 Received parameters from server: [array([[0.37994176, 0.4226613 ]], dtype=float32), array([-0.28427398], dtype=float32)]
📊 Evaluating model


INFO :      Disconnect and shut down


In [3]:
import flwr as fl
import numpy as np

class FLClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return [np.random.rand(2, 2)]  # Return random parameters

    def fit(self, parameters, config):
        print("Training on local data...")
        return parameters, 1, {}

    def evaluate(self, parameters, config):
        print("Evaluating model...")
        return 0.5, 10, {"accuracy": 0.8}  # Dummy loss & accuracy

if __name__ == "__main__":
    fl.client.start_client(
        server_address="127.0.0.1:8080",
        client=FLClient().to_client(),
    )


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)", grpc_status:14, created_time:"2025-03-14T05:55:03.8879266+00:00"}"
>